## 2252938沈诗凡_CNN

### MNIST

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [2]:
# ========== 数据准备 ==========
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# 下载MNIST数据集
dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)

# 划分训练集和验证集（9:1）
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

# 数据加载器
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [3]:
# ========== 定义模型 ==========
# 全连接网络（MLP）
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.flatten = nn.Flatten()
        self.fc = nn.Sequential(
            nn.Linear(28*28, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        x = self.flatten(x)
        return self.fc(x)

# 卷积神经网络（CNN）
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),          # 14x14
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)           # 7x7
        )
        self.fc_layers = nn.Sequential(
            nn.Linear(64*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        x = self.conv_layers(x)
        x = x.view(x.size(0), -1)  # 展平
        return self.fc_layers(x)

In [4]:
# ========== 训练与验证函数 ==========
def train(model, device, train_loader, optimizer, criterion):
    model.train()
    total_loss, correct = 0, 0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        pred = output.argmax(dim=1)
        correct += pred.eq(target).sum().item()
    acc = 100. * correct / len(train_loader.dataset)
    return total_loss / len(train_loader), acc

def evaluate(model, device, val_loader, criterion):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for data, target in val_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            loss = criterion(output, target)
            total_loss += loss.item()
            pred = output.argmax(dim=1)
            correct += pred.eq(target).sum().item()
    acc = 100. * correct / len(val_loader.dataset)
    return total_loss / len(val_loader), acc

In [5]:
# ========== 主训练流程 ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_experiment(model, name, epochs=5):
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    print(f"\n开始训练模型：{name}")
    for epoch in range(1, epochs+1):
        train_loss, train_acc = train(model, device, train_loader, optimizer, criterion)
        val_loss, val_acc = evaluate(model, device, val_loader, criterion)
        print(f"Epoch {epoch}: 训练集 Acc={train_acc:.2f}%  验证集 Acc={val_acc:.2f}%")

    test_loss, test_acc = evaluate(model, device, test_loader, criterion)
    print(f"测试集准确率：{test_acc:.2f}%")
    return test_acc


In [6]:
# ========== 模型对比 ==========
mlp_acc = run_experiment(MLP(), "全连接网络（MLP）")
cnn_acc = run_experiment(CNN(), "卷积神经网络（CNN）")

print("\n📊 性能对比结果：")
print(f"MLP 测试集准确率: {mlp_acc:.2f}%")
print(f"CNN 测试集准确率: {cnn_acc:.2f}%")



开始训练模型：全连接网络（MLP）
Epoch 1: 训练集 Acc=89.12%  验证集 Acc=93.57%
Epoch 2: 训练集 Acc=94.88%  验证集 Acc=95.60%
Epoch 3: 训练集 Acc=96.26%  验证集 Acc=95.87%
Epoch 4: 训练集 Acc=96.99%  验证集 Acc=96.70%
Epoch 5: 训练集 Acc=97.48%  验证集 Acc=96.90%
测试集准确率：97.26%

开始训练模型：卷积神经网络（CNN）
Epoch 1: 训练集 Acc=94.82%  验证集 Acc=98.42%
Epoch 2: 训练集 Acc=98.50%  验证集 Acc=98.37%
Epoch 3: 训练集 Acc=98.98%  验证集 Acc=98.43%
Epoch 4: 训练集 Acc=99.26%  验证集 Acc=98.75%
Epoch 5: 训练集 Acc=99.42%  验证集 Acc=99.02%
测试集准确率：99.10%

📊 性能对比结果：
MLP 测试集准确率: 97.26%
CNN 测试集准确率: 99.10%


### CIFAR-10

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F

In [8]:
# ========= 数据准备 ==========
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# CIFAR-10 数据集（50000训练 + 10000测试）
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

# 划分训练集与验证集（9:1）
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)


In [9]:
# ========= 模型定义 ==========

# 基础CNN
class BaseCNN(nn.Module):
    def __init__(self):
        super(BaseCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),   # 16x16
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2)    # 8x8
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# 模型2：在分类器前再增加一个卷积块
class DeepCNN(nn.Module):
    def __init__(self):
        super(DeepCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(128*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

# 模型3：每个池化前再加一层卷积（更宽）
class WideCNN(nn.Module):
    def __init__(self):
        super(WideCNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(32, 32, 3, padding=1),  # 增加
            nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, padding=1),  # 增加
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64*8*8, 256),
            nn.ReLU(),
            nn.Linear(256, 10)
        )
    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x



# 模型 4: ResNet
# 定义ResNet模型
class BasicBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(BasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        out = F.relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.relu(out)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 16

        self.conv1 = nn.Conv2d(3, 16, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(16)
        self.layer1 = self._make_layer(block, 16, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 64, num_blocks[2], stride=2)
        self.fc = nn.Linear(64, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride):
        layers = []
        layers.append(block(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = torch.nn.functional.adaptive_avg_pool2d(x, (1, 1))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x


In [10]:
# ========= 训练与验证函数 ==========
def train_epoch(model, loader, optimizer, criterion, device):
    model.train()
    total_loss, correct = 0, 0
    for imgs, labels in loader:
        imgs, labels = imgs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        preds = outputs.argmax(1)
        correct += (preds == labels).sum().item()
    acc = correct / len(loader.dataset)
    return total_loss / len(loader), acc

def evaluate(model, loader, criterion, device):
    model.eval()
    total_loss, correct = 0, 0
    with torch.no_grad():
        for imgs, labels in loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            loss = criterion(outputs, labels)
            total_loss += loss.item()
            preds = outputs.argmax(1)
            correct += (preds == labels).sum().item()
    acc = correct / len(loader.dataset)
    return total_loss / len(loader), acc


In [11]:
# ========= 运行实验 ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def run_model(model, name, epochs=10):
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()
    print(f"\n训练模型: {name}")
    for epoch in range(1, epochs+1):
        train_loss, train_acc = train_epoch(model, train_loader, optimizer, criterion, device)
        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        print(f"Epoch {epoch:02d}: 训练Acc={train_acc*100:.2f}%  验证Acc={val_acc*100:.2f}%")
    test_loss, test_acc = evaluate(model, test_loader, criterion, device)
    print(f"{name} 测试集准确率: {test_acc*100:.2f}%")
    return test_acc


In [12]:
# ========= 比较不同模型 ==========
models = {
    "BaseCNN": BaseCNN(),
    "DeepCNN": DeepCNN(),
    "WideCNN": WideCNN(),
    "ResNet": ResNet(BasicBlock, [2, 2, 2])
}

results = {}
for name, model in models.items():
    acc = run_model(model, name, epochs=10)
    results[name] = acc



训练模型: BaseCNN
Epoch 01: 训练Acc=49.30%  验证Acc=59.16%
Epoch 02: 训练Acc=63.83%  验证Acc=67.06%
Epoch 03: 训练Acc=69.64%  验证Acc=68.58%
Epoch 04: 训练Acc=74.32%  验证Acc=70.40%
Epoch 05: 训练Acc=77.99%  验证Acc=71.80%
Epoch 06: 训练Acc=81.35%  验证Acc=72.50%
Epoch 07: 训练Acc=85.15%  验证Acc=71.70%
Epoch 08: 训练Acc=88.58%  验证Acc=72.40%
Epoch 09: 训练Acc=91.56%  验证Acc=73.30%
Epoch 10: 训练Acc=93.89%  验证Acc=72.08%
BaseCNN 测试集准确率: 71.16%

训练模型: DeepCNN
Epoch 01: 训练Acc=50.26%  验证Acc=63.56%
Epoch 02: 训练Acc=66.44%  验证Acc=67.72%
Epoch 03: 训练Acc=73.45%  验证Acc=73.18%
Epoch 04: 训练Acc=78.54%  验证Acc=73.54%
Epoch 05: 训练Acc=83.25%  验证Acc=73.64%
Epoch 06: 训练Acc=87.76%  验证Acc=75.54%
Epoch 07: 训练Acc=91.96%  验证Acc=75.40%
Epoch 08: 训练Acc=94.82%  验证Acc=75.68%
Epoch 09: 训练Acc=96.56%  验证Acc=74.60%
Epoch 10: 训练Acc=97.61%  验证Acc=75.74%
DeepCNN 测试集准确率: 75.60%

训练模型: WideCNN
Epoch 01: 训练Acc=45.97%  验证Acc=56.82%
Epoch 02: 训练Acc=64.02%  验证Acc=67.80%
Epoch 03: 训练Acc=71.45%  验证Acc=71.74%
Epoch 04: 训练Acc=76.86%  验证Acc=73.56%
Epoch 05: 训练Acc=81.44

In [13]:
# ========= 输出最优模型 ==========
best_model = max(results, key=results.get)
print("\n性能对比结果：")
for k, v in results.items():
    print(f"{k}: {v*100:.2f}%")

print(f"\n最优模型: {best_model} ，准确率：{results[best_model]*100:.2f}%")


性能对比结果：
BaseCNN: 71.16%
DeepCNN: 75.60%
WideCNN: 73.33%
ResNet: 75.45%

最优模型: DeepCNN ，准确率：75.60%
